# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [1]:
import numpy as np
import pandas as pd
import random
titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')

## Code the descent algorithm

### Learning Rate: 
Several learning rates (alphas) were tested, and a value of 0.001 was found to be suitable.

### Stopping Policy: 
If the sum of squared errors does not change by more than 0.1% between epochs, we will assume that the algorithm has converged and it will be terminated. This resulted in a training RMSE less than 0.91.

In [18]:
def matrix_factorization(X, U, V, K, epochs=100, alpha=0.001):
    # Transpose the V matrix
    V = V.T
    
    # Initialize e_prev to 0.01 so you are not dividing by 0 on the first iteration
    e_prev = 0.01
    
    # Iterate through each epoch until convergence criteria is met
    for epoch in xrange(epochs):
        
        # Initialize error term for each epoch
        e = 0
        
        # Iterate through each row
        for i in xrange(len(X)):
            # Iterate through each column
            for j in xrange(len(X[i])):
                
                # Only calculate when the cell in the X-matrix is a number greater than 0
                if X[i][j] > 0:
                    eij = X[i][j] - np.dot(U[i,:], V[:,j])
                    
                    # Add squared error to the total error term
                    e = e + eij**2
                    
                    for k in xrange(K):
                        dLdU = -2*eij*V[k][j]
                        dLdV = -2*eij*U[i][k]
                        U[i][k] = U[i][k] - alpha*dLdU
                        V[k][j] = V[k][j] - alpha*dLdV
        
        # Calculate how much the total squared error has changed between epochs
        e_change = abs(e - e_prev)
        # Convert change to percentage
        perc_change = e_change/e_prev
        e_prev = e
        
        # If percent change in error terms between epochs is less than 0.5%, terminate algorithm
        if perc_change < 0.001:
            break
    
    return U, V.T

## 2-Rank Matrix Factorization

### Set up initial guesses for the U and V matrices

In [13]:
# Set n to be number of rows in X
n = len(train_X)
# Set m to be number of columns in X
m = len(train_X[0])

In [14]:
# 2-rank matrix factorization: r = 2
r = 2
# Initialize U and V matrices
U = np.random.random((n,r))
V = np.random.random((m,r))

### Apply 2-rank matrix factorization on the filled training matrix

In [19]:
# Obtain U and V matrices for training set
U_train, V_train = matrix_factorization(train_X, U, V, r)

# Take dot product of U and V.T to get predictions for the training set
X_predTrain = np.dot(U_train, V_train.T)

### Calculate RMSE of this model on the training data

In [20]:
# Initialize SSE and count terms
sse_train = 0
count = 0
for i in xrange(len(train_X)):
    for j in xrange(len(train_X[i])):
        # Only calculate when the cell in the X-matrix is a number greater than 0
        if train_X[i][j] > 0:
            sse_train += (X_predTrain[i][j] - train_X[i][j])**2
            count += 1

# Calculate MSE            
mse_train = sse_train/count
# Calculate RMSE
rmse_train = np.sqrt(mse_train)

0.9041508750401982

### Calculate RMSE of this model on the test data

In [ ]:
# Re-initialize 
r = 2
U = np.random.random((n,r))
V = np.random.random((m,r))

# Obtain U and V matrices for test set
U_test, V_test = matrix_factorization(test_X, U, V, r)

# Take dot product of U and V.T to get predictions for the test set
X_predTest = np.dot(U_test, V_test.T)

In [25]:
print 'Training RMSE\t:\t{}'.format(rmse_train)
print 'Test RMSE\t:\t{}'

Training RMSE	:	0.90415087504
Test RMSE	:	{}


In [58]:
X_pred

array([[ 4.29654443,  3.32455823,  3.19216818, ...,  3.53036536,
         3.72827815,  3.95903894],
       [ 4.1000049 ,  3.14334262,  3.06070519, ...,  3.31105655,
         3.59433504,  3.82655282],
       [ 4.1544517 ,  3.23341066,  3.07720556, ...,  3.45091526,
         3.58136298,  3.79674143],
       ..., 
       [ 3.70723341,  2.85822957,  2.75949611, ...,  3.0256367 ,
         3.22989187,  3.43326231],
       [ 4.02116421,  3.10853607,  2.98904006, ...,  3.29825881,
         3.49301295,  3.71019552],
       [ 3.84150563,  2.99256047,  2.84404927, ...,  3.19635389,
         3.30817504,  3.50620984]])

In [59]:
train_X

array([[  5.,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [  3.,  nan,  nan, ...,  nan,  nan,  nan]])